In [1]:
import structlog
import pandas as pd
import snowflake.connector
import numpy as np
from tqdm import tqdm
import httpx
from datetime import datetime

In [2]:
logger = structlog.get_logger()

In [3]:
def get_snowflake_connection():
    """
    Get Snowflake cursor
    """
    connection = snowflake.connector.connect(
        account="compstak",
        user="periscope",
        password="",
        region="us-east-1",
        role = "sysadmin",
        warehouse="PC_PERISCOPE_WH",
        schema = "PLUGIN_HACKATHON",
        database="GPT",
        autocommit=False,
    )
    return connection

In [4]:
def read_snowflake_query(query: str) -> pd.DataFrame:
    conn = get_snowflake_connection()
    try:
        with conn.cursor() as cursor:
            cursor.execute(query)
            df = cursor.fetch_pandas_all()
            df.columns = map(str.lower, df.columns)
        conn.commit()
    finally:
        conn.close()

    return df

In [5]:
def get_data():
    query = """select * from SAMPLE_DATA"""
    return read_snowflake_query(query)

In [6]:
def standardize_columns(val):
    if pd.isnull(val):
        return np.NaN
    return eval(str(val))

In [7]:
def format_list_as_text(lst):
    if len(lst) == 0:
        return ""
    if len(lst) == 1:
        return lst[0]
    elif len(lst) == 2:
        return ' and '.join(lst)
    else:
        return '{}, and {}'.format(', '.join(lst[:-1]), lst[-1])

In [8]:
def get_nlg(row):
    text = []
    text.append(f"A lease in {row['address']} {row['city']}, {row['state']}, {row['zipcode']} has the following attributes: ")
    if pd.notnull(row['tenant']):
        text.append(f"Tenant Name: {row['tenant']}")
        
    if str(row['tenant_industry']) != "nan":
        ti = format_list_as_text(row['tenant_industry'])
        text.append(f"Tenant Industry: {ti}")
    
    if str(row['commencement_date']) != "nan":
        text.append(f"Lease Commencement Year: {int(row['commencement_date'])}")

    if str(row['expiration_date']) != "nan":
        text.append(f"Lease Expiration Year: {int(row['expiration_date'])}")

    if str(row['tenant_brokerage']) != "nan":
        tb = format_list_as_text(row['tenant_brokerage'])
        text.append(f"Tenants Brokerage Firm: {tb}")
        
    if str(row['tenant_broker']) != "nan":
        tbr = format_list_as_text(row['tenant_broker'])
        text.append(f"Tenants Individual Broker: {tbr}")
        
    if str(row['landlord_brokerage']) != "nan":
        lb = format_list_as_text(row['landlord_brokerage'])
        text.append(f"Landlord's Brokerage Firm: {lb}")
        
    if str(row['landlord_broker']) != "nan":
        lbr = format_list_as_text(row['landlord_broker'])
        text.append(f"Landlord's Individual Broker: {lbr}")
    
    text = '\n'.join(text)
    
    return text


In [9]:
def get_json_to_upsert(master_id,text):
    if pd.isnull(master_id):
        document = {
            "text": text,
            "metadata": {
                "document_id": str(master_id),
                "created_at": str(datetime.utcnow())
            }
        }    
    else:
        document = {
                "text": text,
                "metadata": {
                    "document_id": str(master_id),
                    "url": f"https://enterprise.compstak.com/comps/leases/{master_id}",
                    "created_at": str(datetime.utcnow())
                }
            }
        
    return document

In [10]:
df = get_data()
for col in ["tenant_industry", "tenant_brokerage", "tenant_broker", "landlord_brokerage", "landlord_broker"]:
    df[col] = df[col].apply(standardize_columns)
df["document_info"] = df.apply(get_nlg, axis=1)

In [11]:
upsert_data = df.apply(lambda x: get_json_to_upsert(x["id"], x["document_info"]), axis=1).tolist()

In [12]:
batch_size = 1000
url = "https://chatgpt-retrieval-plugin.stage.cs-int-592.com/upsert"

for i in tqdm(range(0, len(upsert_data), batch_size)):
    json_to_upsert = upsert_data[i:i+batch_size]
    async with httpx.AsyncClient(timeout=None) as client:
        resp = await client.post(url=url, json={"documents": json_to_upsert})
        if resp.status_code != 200:
            logger.info(resp.json)

100%|██████████| 5/5 [00:36<00:00,  7.26s/it]
